In [35]:
import cvxpy as cp
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, CPLEX_CMD
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta


def get_Tz(users):
    """Tz is a list of lists (a list per user) with the timezones in which each user is home and has the possibility to charge.
    A Timezone can be, for instance, (9th of march 10:30, 9th of march 12:45). They are tuples of integer indices."""
    Tz = []
    for user in users:
        availability = user.get('loadprof')
        status = 0
        start = 0
        stop = 0
        Tz_user = []
        for t in range(len(availability)):
            if availability[t] == 1 and status==0:
                start = t
                status = 1
            elif availability[t] == 0 and status==1: #in deze t laadt hij niet meer, maar in de vorige wel nog -> stop = t
                stop = t
                status = 0
                Tz_user.append((start,stop))
            elif availability[t] ==1 and t == (len(availability)-1):
                stop = t+1
                status = 0
                Tz_user.append((start,stop))

        Tz.append(Tz_user)

    return Tz

def get_Z(users):
    """Returns a list of number of times the car is at the chargingstation within simulation period (T) per user."""
    Z = []
    for user in users:
        Z.append(len(user.get("demandprof")))

    return Z


def get_demand(users):
    """Returns a list of demandprofiles per user. The demand profile of a user contains the amount the user wants to charge per timezone Z"""
    demand = []
    for user in users:
        demand.append([round(user.get("demandprof")[z][1]-user.get("demandprof")[z][0],2) for z in range(len(user.get("demandprof")))])
    
    return demand

In [36]:
import pandas as pd
import numpy as np
import os
from random import choice
from time import time


def get_production_consumption(enddatetime = '2017-12-31 23:00:00'):
    startdatetime='2017-01-01 00:00:00'
    df = pd.read_csv(os.path.join('data','Productie en verbruik info Core.csv'), delimiter=';')
    df.Datum = pd.to_datetime(df.Datum + ' ' + df.Tijd, dayfirst=True)
    df.rename(columns={'Datum':'timestamp'}, inplace=True)
    df.drop(['Tijd'], axis = 1, inplace = True)
    df.set_index('timestamp', inplace=True)
    df = df.asfreq('15T')
    df.interpolate(inplace=True)

    return df.loc[startdatetime:enddatetime]


def get_availability_profiles(df):

    df_av = pd.read_excel(os.path.join('data','Laadprofielen.xlsx'), sheet_name='3. State of Charge', header=None)
    # Select rows 4 and onwards and columns D through AS
    df_av = df_av.iloc[3:, 3:45]
    # Reset column names
    df_av.columns = df_av.iloc[0]
    df_av = df_av[1:]

    for column in df_av.columns:
        col_year = list(df_av[column])*52 + list(df_av[column])[:96]
        df[column] = col_year[:len(df)]

    return df


def get_prices(df, dynamic_prices, capaciteitspiek, nb_users):
    df_prices = pd.read_csv(os.path.join('data','BelpexFilter.csv'), delimiter=';')
    df_prices.rename(columns={'Date':'timestamps'}, inplace=True)
    df_prices.timestamps = pd.to_datetime(df_prices.timestamps, dayfirst=True)
    df_prices.sort_values('timestamps', inplace=True)
    df_prices.set_index('timestamps', inplace=True)
    df_prices = df_prices.asfreq('H')
    df_prices = df_prices.resample('15T').interpolate()    
    df_prices.energy_price = (df_prices.energy_price*1e-3 + 0.204*1e-3)*4.3 + 40.4*capaciteitspiek/(365*96*nb_users) 
    df = pd.concat([df, df_prices.loc[df.index[0]:df.index[-1]]], axis=1)
    if dynamic_prices == False:
        df.energy_price = np.mean(df_prices.energy_price)
        print("#############################################################################",'\n',df.energy_price)

    return df


def get_demandprof(user, df):
    """..."""
    demand = []
    Tz = get_Tz([user])[0]
    user['Tz'] = Tz
    for tz in Tz:
        demand.append((df[user.get('rand_profile')+' SOC [kWh]'].iloc[tz[0]], df[user.get('rand_profile')+' SOC [kWh]'].iloc[tz[1]-1]))
        if np.isnan(df[user.get('rand_profile')+' SOC [kWh]'].iloc[tz[0]]) or np.isnan(df[user.get('rand_profile')+' SOC [kWh]'].iloc[tz[1]-1]):
            print(user.get('username'), user.get('rand_profile'), tz[0], tz[1], tz[1]-1)
    return demand


def simulation(users,general):

    dynamic_prices = general.get('dynamic prices')
    capaciteitspiek = general.get('caplimit')
    PV_schaal = general.get('PVschaling')
    charge_rate = general.get('chargerate')
    print('### verbruik en productieprofiel ophalen')
    df = get_production_consumption()
    df['Productie in kW'] = df['Productie in kW']*PV_schaal
    print('### beschikbaarheidsprofielen ophalen')

    df = get_availability_profiles(df)
    print('### prijzen ophalen')

    df = get_prices(df,dynamic_prices, capaciteitspiek, len(users))

    shoppingstations = []
    for user in users:
        user['rand_profile'] = str(user.get("usertype"))+ choice(['A','B','C'])
        user['loadprof'] = df[user.get('rand_profile')]
        user['demandprof'] = get_demandprof(user, df)
        if user.get('usertype') == 7:
            shoppingstations.append(user)

    for ss in shoppingstations:
        loadprofcopy = list(df[ss.get('rand_profile')+' SOC [kWh]'])
        for t in range(len(loadprofcopy)-1):
            if np.isnan(loadprofcopy[t]):
                loadprofcopy[t] = 0
            elif np.isnan(loadprofcopy[t+1]):
                loadprofcopy[t] = 0
            else:
                loadprofcopy[t] = loadprofcopy[t+1]-loadprofcopy[t]
 
        loadprofcopy[-1] = 0

        ss['chargeprof'] = loadprofcopy


    consumptie = np.array(df['Gemeenschappelijk verbruik in kW'])
    productie = np.array(df['Productie in kW'])      
    for t in range(len(df)):
        if capaciteitspiek > consumptie[t] - productie[t] + sum([ss.get('chargeprof')[t] for ss in shoppingstations]):
            consumptie[t] += sum([ss.get('chargeprof')[t] for ss in shoppingstations])
        else:
            diff = consumptie[t] - productie[t] + sum([ss.get('chargeprof')[t] for ss in shoppingstations]) - capaciteitspiek
            diff_per_ss = diff/len(shoppingstations)
            for ss in shoppingstations:
                ss['chargeprof'][t] = max(ss['chargeprof'][t]-diff_per_ss,0)
            consumptie[t] = capaciteitspiek + productie[t]
    for ss in shoppingstations:
        ss['dumb_profile'] = ss.get('chargeprof')
        ss['smart_profile'] = ss.get('chargeprof')
    df['Gemeenschappelijk verbruik in kW'] = consumptie
    df['Productie in kW'] = productie

    users = [user for user in users if user.get('usertype') !=7]

    # print("#### calculating dumb profile ###")
    # users = get_dumb_profiles(users,df, capaciteitspiek,chargeR=charge_rate)
    print("#### calculating smart profile ###")
    return df, users


In [37]:
###########################################################################
### Dit is de file waarin je de parameters voor de simulatie kan kiezen ###
###########################################################################
import pandas as pd
import numpy as np
from datetime import datetime

#inputs: aantal laadpalen, aantal per type, aantal autotype per type user + cap piek


#gebruikers van het type 1
nb_users_type1_no_priority = 10
nb_users_type1_priority =11

#gebruikers van het type 2
nb_users_type2_no_priority = 12
nb_users_type2_priority = 8

#gebruikers van het type 3
nb_users_type3_no_priority = 3
nb_users_type3_priority = 4

#gebruikers van het type 4
nb_users_type4_no_priority = 9
nb_users_type4_priority = 4

#gebruikers van het type 5
nb_users_type5_no_priority =10
nb_users_type5_priority = 15

#gebruikers van het type 6
nb_users_type6_no_priority = 1
nb_users_type6_priority = 1

#gebruikers van het type 7
nb_users_type7_priority =7


#input gegevens van het gebouw 
capaciteitspiek = 25 #[kW] #minstens 22.15, anders kan het standaardverbruik niet altijd geleverd worden. Dit moet minimaal het verbruik van het gebouw zijn.
dynamic_prices = True
PV_schaal = 1

#voor een laadpaal van 22kW is de laadsnelheid per kwartier = 22/4 = 5.5 (kWh --> kWkwartier)
charge_rate = 5.5 #kW per kwartier

#results: duid hieronder aan welke soort documenten u wenst te genereren
pdf = False
excell = False 

#######################################
### HIERONDER NIETS MEER AANPASSEN! ###
#######################################
if pdf == True:
    from weasyprint import HTML
    from jinja2 import Template


systemInfo = {"caplimit":capaciteitspiek,"PVschaling":PV_schaal,"dynamic prices":dynamic_prices,'chargerate':charge_rate}

usernames = [{'type':1,'pr':"", 'nb':nb_users_type1_no_priority, 'priority':0},
             {'type':1,'pr':'_P', 'nb':nb_users_type1_priority, 'priority':1},
             {'type':2,'pr':"", 'nb':nb_users_type2_no_priority, 'priority':0},
             {'type':2,'pr':'_P', 'nb':nb_users_type2_priority, 'priority':1},
             {'type':3,'pr':"", 'nb':nb_users_type3_no_priority, 'priority':0},
             {'type':3,'pr':'_P', 'nb':nb_users_type3_priority, 'priority':1},
             {'type':4,'pr':"", 'nb':nb_users_type4_no_priority, 'priority':0},
             {'type':4,'pr':'_P', 'nb':nb_users_type4_priority, 'priority':1},
             {'type':5,'pr':"", 'nb':nb_users_type5_no_priority, 'priority':0},
             {'type':5,'pr':'_P', 'nb':nb_users_type5_priority, 'priority':1},
             {'type':6,'pr':"", 'nb':nb_users_type6_no_priority, 'priority':0},
             {'type':6,'pr':'_P', 'nb':nb_users_type6_priority, 'priority':1},
             {'type':7,'pr':'_P', 'nb':nb_users_type7_priority, 'priority':2}
             ]
users = []
for username in usernames:
    for nb in range(username.get('nb')):
        users.append({"username":'type'+str(username.get('type'))+'nr'+str(username.get('nb')),"usertype":username.get('type'), "priority":username.get('priority'),'pr':username.get('pr')})


#################
### Simulatie ###
#################
df,users = simulation(users,general=systemInfo)
#dynamische tarieven vs laadcomfort: waarde meegeven



### verbruik en productieprofiel ophalen
### beschikbaarheidsprofielen ophalen
### prijzen ophalen
#### calculating smart profile ###


In [38]:
print(' ### defining problem variables')

# Define problem variables
T = len(df)
Z = get_Z(users)
Z_max = max(Z)
C = len(users)
Tz = get_Tz(users)
demand_brak = get_demand(users)
demand = np.zeros((C,Z_max))
for c in range(C):
    for z in range(Z[c]):
        demand[c,z] = demand_brak[c][z]




for z in Z:
    if np.isnan(z):
        print('Nan in Z')
for user in Tz:
    for tz in user:
        for t in tz:
            if np.isnan(t):
                print('Nan in Tz')
for user in demand:
    for z in user:
        if np.isnan(z):
            print('Nan in demand')
if df.isnull().values.any():
    print('Nan in df')
for user in users:
    if np.isnan(user.get('priority')):
        print('Nan in priority')
    for t in range(T):
        if np.isnan(user.get('loadprof').iloc[t]):
            print('Nan in loadprof')


print(' ### generating model')

model = LpProblem(name='laadpaalstudie', sense=LpMinimize)


zcharge = LpVariable.dicts('zcharge', [(c,z) for c in range(C) for z in range(Z[c])], lowBound=0)
tcharge = LpVariable.dicts('tcharge', [(c,t) for c in range(C) for t in range(T)], lowBound=0, upBound=chargeR)

print(' ### generating objective functions')
obj = lpSum([(demand[c,z] - zcharge[(c,z)])*(users[c].get('priority')/2 +1) for c in range(C) for z in range(Z[c])]) #*(users[c].get('priority')/2 +1)
obj += lpSum((lpSum(tcharge[(c,t)] for c in range(C)) + df['Gemeenschappelijk verbruik in kW'].iloc[t] - df['Productie in kW'].iloc[t])*df['energy_price'].iloc[t] for t in range(T))
model += obj


for t in range(T):
    model += (lpSum(tcharge[(c,t)] for c in range(C)) + df['Gemeenschappelijk verbruik in kW'].iloc[t] - df['Productie in kW'].iloc[t] <= cap)    
    for c in range(C):
        model += (tcharge[(c,t)] == tcharge[(c,t)]*users[c].get('loadprof').iloc[t])
for c in range(C):
    for z in range(Z[c]):
        model += (zcharge[(c,z)] == lpSum([tcharge[(c,t)] for t in range(Tz[c][z][0],Tz[c][z][1])]))

        model += (zcharge[(c,z)] <= demand[c,z])
        

print(' ### solving the problem')

# Solve the problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")


tcharge_arr = np.zeros((C,T))
for c in range(C):
    for t in range(T):
        tcharge_arr[(c,t)] = tcharge[(c,t)].value()

print(' ### inserting smart profile in users')

for c in range(C):
    users[c]['smart_profile'] = [0]*T
    for t in range(T):
        users[c]['smart_profile'][t] = tcharge[(c,t)].value()  


 ### defining problem variables
Nan in df
 ### generating model


NameError: name 'chargeR' is not defined